In [134]:
import requests 
from IPython.display import Image
from bs4 import BeautifulSoup

In [7]:
URL = 'https://www.pagina12.com.ar/'

In [8]:
#para generar una solicitud usamos el modulo requests

In [9]:
pagina = BeautifulSoup(requests.get(URL).text,'lxml')

In [10]:
def parsed_home(soup):
  
    s = soup.find_all('div',attrs={'class':'article-title'})
        
    links=[]
        
    for tag in s:
        link = tag.a['href']
        links.append(link)
    return links 

In [27]:
listado_links = parsed_home(pagina)

In [28]:
r = requests.get(URL)

In [137]:
#Manejo de errores 
errores = []
url_mala = URL.replace('2','3')
try:
    link = requests.get(url_mala)
except Exception as e:
    print('error', e)
    errores.append(e)

error HTTPSConnectionPool(host='www.pagina13.com.ar', port=443): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f8d0c5afb20>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))


In [138]:
url_nota = listado_links[0]
url_nota

'https://www.pagina12.com.ar/316084-el-gobierno-anuncia-al-mediodia-los-detalles-de-las-nuevas-m'

In [154]:
#TODO: pasar todo esto a una funcion!
try:
    url_p = requests.get(url_nota)
    if url_p.status_code == 200:
        url_bs = BeautifulSoup(url_p.text,'lxml')
        #print(url_bs.prettify())
        contenido = url_bs.find('div',attrs={'class':'article'})
        
        #guardar todo lo extraido en el diccionario! 
        ret_dic = {}
    
        #2 posibles formas de extraer la fecha de la pagina    
        
        fecha = contenido.find('div',attrs={'class':'time'}).span.text
        #o se puede extraer la fecha como
        # .get('datetime') para guardar en formato yyyy-mm-dd
        fecha01 = contenido.find('span',attrs={'pubdate':'pubdate'}).get('datetime')
        if fecha01:
            ret_dic['fecha'] = fecha01
        else:
            ret_dic['fecha'] = None
        
        
        titulo = contenido.h1.text
        if titulo:
            ret_dic['Titulo'] = titulo
        else:
            ret_dic['Titulo'] = None
        
        bajada = contenido.find('div',attrs={'class':'article-summary'}).text
        if bajada:
            ret_dic['bajada'] = bajada
        else:
            ret_dic['bajada'] = None
        
        
        #extraccion del cuerpo
        cuerpo = contenido.find_all('p',attrs={'class': None})
        cuerpo_pag = []
        for c in cuerpo:
            cuerpo_pag.append(c.text)
        
        if cuerpo_pag:
            ret_dic['cuerpo'] = cuerpo_pag
        else:
            ret_dic['cuerpo'] = None
        
        #extraccion de las imagenes
        links_media = []
        media = contenido.find('div',attrs={'class':'article-main-media-image'}).find_all('img')
        len_media = len(media)
        
        if len_media == 0:
            print('no hay imagenes')
        else:
            for link in media:
                links_media.append(link['data-src'])
        #validar si el link a elegir da una resp=uesta de 200
        img_source = links_media[-1]
        img_request = requests.get(img_source)
        
        if img_request.status_code == 200:
            #imagen = Image(img_request.content)
            ret_dic['imagen'] = img_source
        
        
except Exception as e:
    print(e)

In [ ]:
#unificando el codigo para cuando se genere la nota
def scrap_note(url):
    try:
        nota = requests.get(url)
    except Exception as e:
        print('Error scrapeando url', url)
        print(e)
        return None
    
    if nota.status_code == 200:
        print(f'Error en obtener la pagina {url}')
        print(f'Status code {nota.status_code}')
        return None
    
    s_nota = BeautifulSoup(nota.text,'lxml')
    ret_dic = #nombre de funcion obtenerInfo(s_nota)
    ret_dic['url'] = url
    return ret_dic